### Adjust Word Group Repeat With Youtube Link

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

In [ ]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/3-Adjust Word Group Repeat With Youtube Link"

#Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
path_folder_file = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish/Twogram_Threegram_Selected.xlsx"
sheet = "twogram_threegram_selected"  # Sheet1

In [ ]:
#df_word_group_time_loc = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

In [ ]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

In [ ]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

In [ ]:
## All word group result convert to join result in one row
#df_result_repeat = pd.DataFrame()
#for word_group in search_list:
#    # for repeat 1
#    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
#    # for repeat 2
#    try:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
#    except:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
#    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
#    # repeat block
#    for i in range(len(df_word_group_search_repeat1)):        
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)
#
#    try:
#        df_link_default_var = df_link_default
#        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
#        if word_time_diff_var < 4:
#            word_time_diff_var2 = 3
#        else:
#            word_time_diff_var2 = word_time_diff_var+1.0
#
#        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
#    except:
#        pass
#
#    for j in range(3):
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
#        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
#
#df_result_repeat.reset_index(drop=True, inplace=True)
#df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])
#
#df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time2.xlsx", index=False)
#df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join2.xlsx", index=False)

In [ ]:
#df_result_repeat

In [ ]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [ ]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join.xlsx", index=False)


In [ ]:
df_all_join_result

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel("Twogram Threegram Youtube Link Selected.xlsx")
df

In [ ]:
df_count = df.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

In [ ]:
df_count_merge = pd.merge(df, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

In [ ]:
df_count_merge["search_string"].nunique()

In [ ]:
df_count_merge[df_count_merge["count"] == 1]

In [ ]:
df_count_merge.to_excel("Twogram Threegram Youtube Link Selected_Count.xlsx", index=False)

In [ ]:
df_word_group_time_loc_manuel = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Count_Manuel.xlsx")
df_word_group_time_loc_manuel.sort_values(by=["count","search_string"],inplace=True, ascending=False)
df_word_group_time_loc_manuel.reset_index(drop=True, inplace=True)
df_word_group_time_loc_manuel

In [ ]:
df_word_group_time_loc_manuel.to_excel("Twogram Threegram Youtube Link Selected_Manuel.xlsx", index=False)